# Homework 5 "Putting them all together" Solution

Solve the following boundary value problem using the "standard" (i.e., Ritz-Galerkin-Courant) finite element method.

\begin{align*}
\nabla \cdot (a(\mathbf{x})\nabla u) &= 0 \quad \text{in} \quad \Omega, \\
u &= 10 \quad \text{on} \quad \Gamma_{g1} \\
u &= 0 \quad \text{on} \quad \Gamma_{g2}.
\end{align*}

<img src="./Figures/domain.png" width="480">

The problem domain ($\Omega$) is the gray-shaded area in the above figure and the two Dirichlet boundaries ($\Gamma_{g1}$ and $\Gamma_{g2}$) are marked by bold lines. $a(\mathbf{x})$ in the above equation is a spatially variable coefficient given as 2 in the dark gray region and 1 in the light gray region. On the rest of the domain boundary, it is assumed that $\partial u/\partial n = 0$, where $n$ is the distance along the outward normal direction on the boundary.


## Load basic modules

In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.tri as mtri

## Discretization Parameters

In [ ]:
nnode = 42
nelem = 40
ndim = 2
NODES_PER_ELEMENT = 3 # P1 element used.

## Arrays

### Coordinates and connectivity

In [ ]:
coord = np.zeros((nnode,ndim))
connectivity = np.zeros((nelem,3),dtype=int)

In [ ]:
for j in range(7):
    for i in range(6):
        node = i + j * 6
        coord[node,0] = i*1.0
        coord[node,1] = j*1.0


In [ ]:
# Bottom 10 elements, numbered from right to left
connectivity[9,:] = [0,1,6]
connectivity[8,:] = [1,7,6]
connectivity[7,:] = [1,2,7]
connectivity[6,:] = [2,8,7]
connectivity[5,:] = [2,3,8]
connectivity[4,:] = [3,9,8]
connectivity[3,:] = [3,4,9]
connectivity[2,:] = [4,10,9]
connectivity[1,:] = [4,5,10]
connectivity[0,:] = [5,11,10]
# 8 elements in the left column, numbered bottom up
connectivity[10,:] = [6,7,12]
connectivity[11,:] = [7,13,12]
connectivity[12,:] = [12,13,18]
connectivity[13,:] = [13,19,18]
connectivity[14,:] = [18,19,24]
connectivity[15,:] = [19,25,24]
connectivity[16,:] = [24,25,30]
connectivity[17,:] = [25,31,30]
# Top 10 elements, numbered from left to right
connectivity[18,:] = [30,31,36]
connectivity[19,:] = [31,37,36]
connectivity[20,:] = [31,32,37]
connectivity[21,:] = [32,38,37]
connectivity[22,:] = [32,33,38]
connectivity[23,:] = [33,39,38]
connectivity[24,:] = [33,34,39]
connectivity[25,:] = [34,40,39]
connectivity[26,:] = [34,35,40]
connectivity[27,:] = [35,41,40]
# 6 elements in the right column, numbered top down
connectivity[28,:] = [29,35,34]
connectivity[29,:] = [28,29,34]
connectivity[30,:] = [23,29,28]
connectivity[31,:] = [22,23,28]
connectivity[32,:] = [17,23,22]
connectivity[33,:] = [16,17,22]
# 4 elements in the central horizontal row, numbered right to left
connectivity[34,:] = [16,22,21]
connectivity[35,:] = [15,16,21]
connectivity[36,:] = [15,21,20]
connectivity[37,:] = [14,15,20]
# 2 elements in the cnetral end
connectivity[38,:] = [20,21,26]
connectivity[39,:] = [21,27,26]


### Gradients of basis functions
Since two-dimensional P1 elements are used, the basis functions on three nodes of the reference triangle are given as
\begin{align}
\hat{\phi}_0 &= 1 - \xi - \eta \\
\hat{\phi}_1 &= \xi \\
\hat{\phi}_2 &= \eta
\end{align}

The gradient of $\hat{\phi}_{i}$ is given as
$\nabla \hat{\phi}_{i} = (\partial \hat{\phi}_{i}/\partial \xi, \partial \hat{\phi}_{i}/\partial \eta)$ and their values are constant in the element. So, they are stored as static arrays rather than a function to be evaluated at quadrature nodes.

In [ ]:
dphihat = np.zeros((NODES_PER_ELEMENT,ndim))
dphihat[0,:] = np.array([-1.0,-1.0])
dphihat[1,:] = np.array([1.0,0.0])
dphihat[2,:] = np.array([0.0,1.0])

### Other helper arrays
`bcflag` array stores integer flags for all the nodes in the domain such that 0 for non-boundary nodes and 1 for boundary nodes.

`bcval` array stores Dirichlet boundary conditions for all the nodes such that 0.0 for non-Dirichlet nodes and given values (as the problem specifies) for nodes on the Dirichlet boundaries.

`a_coeff` array stores coefficient $a(\mathbf{x})$ for all the elements. In principle, they need to provide the coefficient values for all the quadrature nodes but P1 elements used for this problem has a single quadrature node per element (i.e., centroid). Note that the way elements are numbered made it easy to assign different values of the coefficient as the problem specifies: 1.0 for element number 0 to 19; 2.0 for element number >= 20.

In [ ]:
bcflag = np.zeros((nnode,1),dtype=int)
bcflag[5]  = 1
bcflag[11] = 1
bcflag[26] = 1
bcflag[27] = 1

bcval = np.zeros((nnode,1))
bcval[5]  =  0.0
bcval[11] =  0.0
bcval[26] = 10.0
bcval[27] = 10.0

a_coeff = np.ones(nelem)
a_coeff[np.arange(nelem)>=20] = 2.0
#print(a_coeff)

### Partial verification of the arrays created

In [ ]:
plt.triplot(coord[:,0], coord[:,1], connectivity, 'k-o')
plt.gca().set_aspect('equal')

## Constructing Stiffness Matrix and Force Vector

### Element-wise calculation and assembly

An element stiffness matrix is given as
\begin{equation}
k_{ij} = \int_{\hat{T}} \hat{a} \left( \nabla \hat{\phi}_{i} \cdot \nabla \hat{\phi}_{j} \right) \,J\,d\hat{\Omega}
\end{equation}

The integration is exactly evaluated by 1-point quadrature at $(\xi,\eta)=(\frac{1}{3},\frac{1}{3})$ with the weight of $\frac{1}{2}$:
\begin{align}
k_{ij} &= \frac{1}{2}\, \hat{a}\, \left( \nabla \hat{\phi}_{i} \cdot \nabla \hat{\phi}_{j} \,J \right)_{(\xi,\eta)=(1/3,1/3)} \\
       &= \frac{1}{2}\, \hat{a}\, \left( \frac{\partial \hat{\phi}_{i}}{\partial \xi} \frac{\partial \hat{\phi}_{j}}{\partial \xi} + \frac{\partial \hat{\phi}_{i}}{\partial \eta} \frac{\partial \hat{\phi}_{j}}{\partial \eta} \right) J
\end{align}

Note that the partial derivatives of the basis functions are all constants. 

Jacobian for P1 is equal to $\det(B_{T})$ or the ratio of an element's area to the reference element's: $A/A_{\text{ref}}$. However, since all the elements in the chosen discretization have the same area with the reference triangle, $J = 1.0$ for all of them.


In [ ]:
K = np.zeros((nnode,nnode))
F = np.zeros((nnode,1))
for e in range(nelem):
    ## Although not needed for this problem, 
    ## the Jacobian calculation is shown here
    x0 = coord[connectivity[e,0],:]
    x1 = coord[connectivity[e,1],:]
    x2 = coord[connectivity[e,2],:]
    B_T = np.matrix([[x1[0]-x0[0],x2[0]-x0[0]],[x1[1]-x0[1],x2[1]-x0[1]]])
    J = np.linalg.det(B_T)
    #print(x0,B_T,J)

    ## element arrays are created
    k_matrix = np.zeros((NODES_PER_ELEMENT,NODES_PER_ELEMENT))
    f_vector = np.zeros(NODES_PER_ELEMENT)

    ## Compute element k. 
    # We don't do anything yet for element force vector
    # because no source term was given.
    for i in range(NODES_PER_ELEMENT):
        for j in range(NODES_PER_ELEMENT):
            for d in range(ndim):
                k_matrix[i,j] += dphihat[i,d]*dphihat[j,d]
            k_matrix[i,j] *= 0.5*J*a_coeff[e]

    ## Deal with Dirichlet BC
    # If i-th node is Dirichlet, overwrite the i-th row of k matrix
    # with the i-th row of the identity matrix; and i-th element of
    # f vector with the corresponding bcval.
    for i in range(NODES_PER_ELEMENT):
        gI = connectivity[e,i]
        if bcflag[gI] == 1:
            #print(e,i,gI,bcval[gI])
            k_matrix[i,:] = 0.0
            k_matrix[i,i] = 1.0
            f_vector[i] = bcval[gI]
            #print(k_matrix,f_vector)

    ## Assemble the global matrices by mapping the local node numbers 
    ## to the global counterparts.
    for i in range(NODES_PER_ELEMENT):
        gI = connectivity[e,i]
        for j in range(NODES_PER_ELEMENT):
            gJ = connectivity[e,j]
            K[gI,gJ] += k_matrix[i,j]
        F[gI] += f_vector[i]
    #print(k_matrix, f_vector)

### Partial verification of K and F

Check for node 11 and 27, the non-shared node at the lower right and the central end of the domain, respectively.

In [ ]:
# 12-th element of K[11,:] should be 1 and all the others should be 0.
# 12-th element of F should be 0, the prescribed value by the Dirichlet BC.
print(K[11,:], F[11])

# 28-th element of K[27,:] should be 1 and all the others should be 0.
# 28-th element of F should be 10, the prescribed value by the Dirichlet BC.
print(K[27,:], F[27])

## Solve the linear system

In [ ]:
solution = la.solve(K,F)
#print(solution)

## Visualize the solution

If the given problem is viewed as a static heat conduction problem, it should be straightforward to predict the solution. From the high temperature end to the low-temperature end, a piecewise linear temperature profile will be set up. 

In the high conductivity ($a=2.0$) domain, a low temperature gradient is set up while the low-conductivity ($a=1.0$) domain has a high temperature gradient. This is the result from the requirement that the heat flow should be constant when the coefficient is constant and the heat flow ($a\nabla T$) should be continuous at the interface between the domains of different conductivity.

**Is there anything else that is consistent or inconsistent with your physical intuition?**

In [ ]:
# Plot in a meshgrid (Visualization second method)
fig, ax = plt.subplots(figsize=(6,5))
ax.set_aspect('equal')
plt.tricontourf(coord[:,0], coord[:,1], connectivity, solution[:,0], levels=41, cmap='inferno', vmin=0, vmax=10)
plt.triplot(coord[:,0], coord[:,1], connectivity, 'ko-')
plt.title('Solution')
plt.colorbar(ticks=np.linspace(0,10,11))
plt.show ()